In [6]:
import sys
sys.path.append("/home/zhouqua1") 
sys.path.append("/home/zhouqua1/NCPOP") 
from inputlds import*
from functions import*
from ncpol2sdpa import*
import numpy as np
import pandas as pd
from math import sqrt
from sympy.physics.quantum import HermitianOperator, Operator



def data_generation(g,f_dash,proc_noise_std,obs_noise_std,T):
    # hidden state
    n=len(g)
    # Obervation 
    m=len(f_dash)
    ds1 = dynamical_system(g,np.zeros((n,m)),f_dash,np.zeros((m,m)),
            process_noise='gaussian',
            observation_noise='gaussian', 
            process_noise_std=proc_noise_std, 
            observation_noise_std=obs_noise_std)
    inputs = np.zeros((m,T))
    h0=np.ones(ds1.d) # initial state
    ds1.solve(h0=h0, inputs=inputs, T=T) 
    return np.asarray(ds1.outputs).reshape(T,m).tolist()


#Produce 2-dim observation data
n=3
m=2
T=20
g1 = np.matrix([[0.8,0,0],[0,0.9,0],[0,0,0.1]])
f_dash1 = np.matrix([[1.0,0.5,0.3],[0.1,0.1,0.1]])
g2=np.random.randint(0, 2, (n,n))
f_dash2 = np.random.randint(0, 2, (m,n))
proc_noise_std=0.01
obs_noise_std=0.01
patient_n1= 3
patient_n2= 2
D = np.zeros((T,m))
Dataa=pd.DataFrame(range(T),columns=['Time'])
#np.asarray(ds1.outputs).reshape(-1).tolist()
for n in range(patient_n1+patient_n2):
    if n < patient_n1:
        D=data_generation(g1,f_dash1,proc_noise_std,obs_noise_std,T)
        Data=pd.DataFrame(D,columns=['Patient_'+str(n+1)+'_HBP', 'Patient_'+str(n+1)+'_BloodPressure'])
    else:
        D=data_generation(g2,f_dash2,proc_noise_std,obs_noise_std,T)
        Data=pd.DataFrame(D,columns=['Patient_'+str(n+1)+'_HBP', 'Patient_'+str(n+1)+'_BloodPressure'])
    Dataa=pd.concat([Dataa, Data], axis=1)
#Data=pd.merge(Dataa1, Dataa2, on='Time')
HBP_Data = Dataa.loc[:,Dataa.columns.str.contains('HBP')]
BP_Data = Dataa.loc[:,Dataa.columns.str.contains('BloodPressure')]
#print(g1,f_dash1,g2,f_dash2,HBP_Data,BP_Data)
print([0, 0, 0, 1, 1])
#len(HBP_Data)

[0, 0, 0, 1, 1]


In [7]:
class NCPOLR(object):
    """Estimator based on NCPOP Regressor

    References
    ----------
    Quan Zhou https://github.com/Quan-Zhou/Proper-Learning-of-LDS/blob/master/ncpop/functions.py
    
    Examples
    --------
    """
    
    def __init__(self, **kwargs):
        super(NCPOLR, self).__init__()
    
    
    def generate_multioperators(self, name, n_vars, m_vars, hermitian=None, commutative=False):
        """Generates a number of commutative or noncommutative operators

        :param name: The prefix in the symbolic representation of the noncommuting
                     variables. This will be suffixed by a number from 0 to
                     n_vars-1 if n_vars > 1.
        :type name: str.
        :param n_vars: The number of variables.
        :type n_vars: int.
        :param hermitian: Optional parameter to request Hermitian variables .
        :type hermitian: bool.
        :param commutative: Optional parameter to request commutative variables.
                            Commutative variables are Hermitian by default.
        :type commutative: bool.

        :returns: list of :class:`sympy.physics.quantum.operator.Operator` or
                  :class:`sympy.physics.quantum.operator.HermitianOperator`
                  variables

        :Example:

        >>> generate_variables('y', 2, commutative=True)
        #[y0, y1]
        """

        variables = []
        variables1 = []
        variables2 = []
        for i in range(n_vars):
            if n_vars > 1:
                var_name1 = '%s%s' % (name, i)
            else:
                var_name1 = '%s' % name
            if hermitian is not None and hermitian:
                variables1.append(HermitianOperator(var_name1))
            else:
                variables1.append(Operator(var_name1))
            variables1[-1].is_commutative = commutative
        for n in range(len(variables1)):        
            for j in range(m_vars):
                if m_vars > 1:
                    var_name = '%s%s' % (variables1[n], j)
                else:
                    var_name = '%s' % variables1[n]
                if hermitian is not None and hermitian:
                    variables2.append(HermitianOperator(var_name))
                else:
                    variables2.append(Operator(var_name))
                variables2[-1].is_commutative = commutative
        var = np.matrix(np.array(variables2).reshape(n_vars,m_vars))
        var = np.array(variables2).reshape(n_vars,m_vars)
        #print(variables1,variables2)
        return var

        
    def estimate(self, Y, m,level):
        """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals.
        The module converts a noncommutative optimization problem provided in SymPy
        format to an SDPA semidefinite programming problem.
        Define a function for solving the NCPO problems with 
        given standard deviations of process noise and observtion noise,
        length of  estimation data and required relaxation level. 

        Parameters
        ----------
        Y: array
            Variable seen as input
        m:
        n:

        Returns
        -------
        y_predict: array
            regression predict values of y or residuals
        obj: num
            Objective value in optima
        """
        n=7
        T= len(Y)
        Y = np.transpose(Y)
        # Decision Variables
        G = NCPOLR().generate_multioperators("G", n_vars=n, m_vars=n, hermitian=True, commutative=False)
        Fdash = NCPOLR().generate_multioperators("Fdash", n_vars=m, m_vars=n, hermitian=True, commutative=False)
        phi = NCPOLR().generate_multioperators("phi", n_vars=n, m_vars=T+1, hermitian=True, commutative=False)
        q = NCPOLR().generate_multioperators("q", n_vars=n, m_vars=T, hermitian=True, commutative=False)
        p = NCPOLR().generate_multioperators("p", n_vars=m, m_vars=T, hermitian=True, commutative=False)
        f = NCPOLR().generate_multioperators("f", n_vars=m, m_vars=T, hermitian=True, commutative=False)
        

        # Objective
        #obj = sum((Y[i]-f[i])**2 for i in range(T)) + 0.0005*sum(p[i]**2 for i in range(T)) + 0.0001*sum(q[i]**2 for i in range(T))
        obj = sum((Y[mm][i])*2 for i in range(T) for mm in range(m))+ 0.0005*sum(p[mm][i]*2 for i in range(T) for mm in range(m)) + 0.0001*sum(q[nn][i]*2 for i in range(T) for nn in range(n)) 
        
        # Constraints
        
        ine1 = [f[mm][i] - Fdash[mm][nn]*phi[nn][i+1] - p[mm][i] for nn in range(n) for i in range(T) for mm in range(m)]
        ine2 = [-f[mm][i] + Fdash[mm][nn]*phi[nn][i+1] + p[mm][i] for nn in range(n) for i in range(T) for mm in range(m)]
        ine3 = [phi[nn][i+1] - G[nn][nn]*phi[nn][i] - q[nn][i] for nn in range(n) for i in range(T)]
        ine4 = [-phi[nn][i+1] + G[nn][nn]*phi[nn][i] + q[nn][i] for i in range(T) for nn in range(n)]
        #ine5 = [(Y[i]-f[i])**2 for i in range(T)]
        ines = ine1+ine2+ine3+ine4 

        # Solve the NCPO
        GG = Operator(np.asarray(G).reshape(-1))
        FFdash = Operator(np.asarray(Fdash).reshape(-1))
        f = Operator(np.asarray(f).reshape(-1))
        q = Operator(np.asarray(q).reshape(-1))
        p = Operator(np.asarray(p).reshape(-1))
        phi = Operator(np.asarray(phi).reshape(-1))
        #print([Operator(GG),Operator(FFdash),Operator(f),Operator(p),Operator(phi),Operator(q)])
    
        sdp = SdpRelaxation(variables = flatten([GG,FFdash,f,p,phi,q]),verbose = 1)
        sdp.get_relaxation(level, objective=obj, inequalities=ines)
        sdp.solve(solver='mosek')
        '''
        Attributes:
          - `monomial_sets`: The monomial sets that generate the moment matrix blocks.

          - `monomial_index`: Dictionary that maps monomials to SDP variables.

          - `constraints`: The complete set of constraints after preprocesssing.

          - `primal`: The primal optimal value.

          - `dual`: The dual optimal value.

          - `x_mat`: The primal solution matrix.

          - `y_mat`: The dual solution matrix.

          - `solution_time`: The amount of time taken to solve the relaxation.

          - `status`: The solution status of the relaxation.
        '''
        sdp.write_to_file("slutions.csv")
        sdp.write_to_file('example.dat-s')
        sdp.find_solution_ranks()
        #sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:/Users/zhouq/Documents/sdpa7-windows/sdpa.exe"})
        print(sdp.primal, sdp.dual,sdp.status)
        return sdp.primal
        '''
        if(sdp.status != 'infeasible'):
            print('ok.')
            f_pred = []
            p_pred = []
            phi_pred = []
            q_pred = []
            for t in range(len(np.asarray(p).reshape(-1))):
                f_pred.append(sdp[f[t]])
                p_pred.append(sdp[p[t]])
            for t in range(len(np.asarray(q).reshape(-1))):
                phi_pred.append(sdp[phi[t]])
                q_pred.append(sdp[q[t]])
            Para = [sdp[GG].reshape(n,n),sdp[FFdash].reshape(m,n)]
            return f_pred.reshape(m,T),p_pred.reshape(m,T),phi_pred.reshape(n,T),p_pred.reshape(n,T),Para
            '''


In [8]:
from itertools import product
X=HBP_Data
T= len(X)
#Y = np.transpose(test_tsdata)
m= len(np.transpose(X))
print(T,m)
K = 2
#L= np.array(np.random.randint(0,K,m))           
L = [1,1,0,0,0] 
level = 1
X_Group0 = X.iloc[:,np.flatnonzero(L)]
X_Group0 = np.array(X_Group0)#.reshape(-1)
N0 = len(np.transpose(X_Group0))

X_Group1 = X.iloc[:,np.flatnonzero(list(abs(np.asarray(L) -1)))]
X_Group1 = np.array(X_Group1)#.reshape(-1)
N1 = len(np.transpose(X_Group1))
print(N0,N1)
primal_value = NCPOLR().estimate(X_Group0,N0,level)+ NCPOLR().estimate(X_Group1,N1,level)

#y_LDS2_predict = NCPOLR().estimate(list(y_LDS2),level)
print(print(L,primal_value))

# itaration
l = [0,1]
prod = list(product(l, repeat=m))

for ll in range(len(prod)):
    L_new= np.array(prod[ll])
    if len([i for i in L_new if i==1]) >1 and len([i for i in L_new if i==0]) >1:
        #Separate into K=2 groups
        X_Group00 = X.iloc[:,np.flatnonzero(L_new)]
        X_Group00 = np.array(X_Group00)#.reshape(-1)
        N00 = len(np.transpose(X_Group00))

        X_Group11 = X.iloc[:,np.flatnonzero(list(abs(np.asarray(L_new) -1)))]
        X_Group11 = np.array(X_Group11)#.reshape(-1)
        N11 = len(np.transpose(X_Group11))
        print(X_Group00,X_Group11)

        primal_value_new = NCPOLR().estimate(X_Group00,N00,level)+ NCPOLR().estimate(X_Group11,N11,level)
        print(L_new,primal_value_new)
        if primal_value_new < primal_value:
            L= L_new
            primal_value = primal_value_new 
            print(L,primal_value)
        else:
            continue
    else:
        continue
#return L
        



20 5
2 3
The problem has 6 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 90
Generating moment matrix...
Reduced number of SDP variables: 90 90 (done: 101.11%, ETA 00:00:-0.0)
Processing 137/840 constraints...

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

...

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

Processing 1120/1120 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 642411)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep

Processing 840/840 constraints...
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 90              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 364231)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep. 

In [ ]:
from matplotlib import pyplot as plt

y_1 = list(190+70*HBP_Data['Patient_1_HBP'])
y_2 = np.asarray(190+70*HBP_Data['Patient_2_HBP'])
y_3 = list(190+70*HBP_Data['Patient_3_HBP'])
y_4 = np.asarray(190+70*HBP_Data['Patient_4_HBP'])
y_5 = list(190+70*HBP_Data['Patient_5_HBP'])
x = list(Dataa['Time'])

plt.figure(figsize=(20,8),dpi=80)

plt.plot(x,y_1,label="Patient_1_PRO",
         color="orange",
         linestyle=':',
         linewidth=5,
         alpha=0.8)

plt.plot(x,y_2,label="Patient_2_PRO",
         color="cyan",
         linestyle=':',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_3,label="Patient_3_PRO",
         color="pink",
         linestyle=':',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_4,label="Patient_4_PRO",
         color="blue",
         linestyle=':',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_5,label="Patient_5_PRO",
         color="green",
         linestyle=':',
         linewidth=5,
         alpha=0.8)

_xtick_labels = ["Day{}".format(i) for i in x]
plt.xticks(x,_xtick_labels )

plt.grid(alpha=0.4)

plt.legend(loc="upper left")

plt.show()

In [ ]:
from matplotlib import pyplot as plt

y_1 = list(100+500*BP_Data['Patient_1_BloodPressure'])
y_2 = list(100+500*BP_Data['Patient_2_BloodPressure'])
y_3 = list(100+500*BP_Data['Patient_3_BloodPressure'])
y_4 = list(100+500*BP_Data['Patient_4_BloodPressure'])
y_5 = list(100+500*BP_Data['Patient_5_BloodPressure'])
x = list(Dataa['Time'])

plt.figure(figsize=(20,8),dpi=80)

plt.plot(x,y_1,label="Patient_1_SBP",
         color="orange",
         linestyle='-.',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_2,label="Patient_2_SBP",
         color="cyan",
         linestyle='-.',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_3,label="Patient_3_SBP",
         color="pink",
         linestyle='-.',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_4,label="Patient_4_SBP",
         color="blue",
         linestyle='-.',
         linewidth=5,
         alpha=0.8)
plt.plot(x,y_5,label="Patient_5_SBP",
         color="green",
         linestyle='-.',
         linewidth=5,
         alpha=0.8)

_xtick_labels = ["Day{}".format(i) for i in x]
plt.xticks(x,_xtick_labels )

plt.grid(alpha=0.4)

plt.legend(loc="upper left")

plt.show()